In [290]:
!echo Installing bbSearch module from web ...
!echo creating bbmodcache subfolder
!mkdir -p bbmodcache
!echo downloading bbSearch module
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py

from bbmodcache.bbSearch import SearchProblem, search

Installing bbSearch module from web ...
creating bbmodcache subfolder
downloading bbSearch module
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18767  100 18767    0     0   352k      0 --:--:-- --:--:-- --:--:--  352k


Code from Search exercise 7. Creates the grids with their respective blocks and their respective colours

In [291]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from copy import deepcopy

plt.ioff()  ## Turn off immediate display of plots

COLORS = ["lightgray", "red", "blue", "green", "yellow",
          "orange", "purple", "pink", "brown"]

class BlockState:

      # Specify mapping from directions to grid coordinate offsets:
      neighbour_offset = {"left": (0,-1), "right": (0,+1), "down":(+1,0), "up":(-1,0)}

      def __init__( self, blockstate, colors=COLORS ):
        self.blockstate = blockstate
        self.nrows = len(blockstate)
        self.ncols = len(blockstate[0])
        self.blocknums = set().union(*[set(row) for row in blockstate])
        self.blocknums = self.blocknums - {0}
        self.blocknumlist = list(self.blocknums)
        self.colors = colors

      def __repr__(self):
        return( str( self.blockstate ))

      # Find the cells occupied by a given number
      def blockcells( self, blocknum ):
          blockcells = []
          for row in range(self.nrows):
            for col in range(self.ncols):
              if self.blockstate[row][col] == blocknum:
                blockcells.append((row,col))
          return blockcells

      # Test if a cell is free (unblocked) in a given direction
      # Free if not blocked by edge of grid or by a cell of different colour
      def free_cell( self, direction, cell ):
        row, col = cell
        offrow, offcol = BlockState.neighbour_offset[direction]
        neighrow, neighcol = (row + offrow, col + offcol)
        if not (0 <= neighrow < self.nrows): return False #at top or bottom
        if not (0 <= neighcol < self.ncols): return False #at left or right
        neighval = self.blockstate[neighrow][neighcol]
        # Neighboring cell must be empty or part of the same coloured block
        return  (neighval==0 or neighval==self.blockstate[row][col])

      def free_block( self, direction, blockn ):
          blockcells = self.blockcells(blockn)
          for cell in blockcells:
            if not self.free_cell(direction, cell):
              return False
          return True

      def possible_moves(self):
        moves = []
        for blocknum in self.blocknumlist:
          for direction in ["left", "right", "down", "up"]:
              if self.free_block(direction, blocknum):
                  moves.append((blocknum, direction))
        return moves

      def next_state(self, move):
          next_blockstate = deepcopy(self.blockstate)
          blockno, direction = move
          cells = self.blockcells(blockno)
          ## first clear all cells of the block (set to 0)
          for cell in cells:
            row, col = cell
            next_blockstate[row][col] = 0
          rowoff, coloff = BlockState.neighbour_offset[direction]
          ## now set all neighbour cells (in move direction) to be
          ## cells with the blocknumber
          for cell in cells:
            row, col = cell
            next_blockstate[row+rowoff][col+coloff] = blockno
          return BlockState(next_blockstate)

      def color_key(self):
          return {b:self.colors[b] for b in self.blocknumlist}

      def figure(self, scale=0.5):
          nrows = self.nrows
          ncols = self.ncols
          fig, ax = plt.subplots(figsize=(ncols*scale+0.1,nrows*scale+0.1))
          plt.close(fig)
          ax.set_axis_off() # Don't show border lines and coordinate values

          frame = patches.Rectangle((0,0),1,1, linewidth=5, edgecolor='k', facecolor='w')
          ax.add_patch(frame)

          for row in range(nrows):
            for col in range(ncols):
                greyrect = patches.Rectangle( (((col*0.9)/ncols)+0.05,
                                               (((nrows-row-1)*0.9)/nrows)+0.05 ),
                                            0.9/ncols, 0.9/nrows,
                                            linewidth=1, edgecolor="gray", facecolor="lightgray")
                ax.add_patch(greyrect)

          for row in range(nrows):
            for col in range(ncols):
                cellval = self.blockstate[row][col]
                if cellval > 0:
                  cellcol = COLORS[cellval]
                  rect = patches.Rectangle( (((col*0.9)/ncols)+0.05,
                                             (((nrows-row-1)*0.9)/nrows)+0.05 ),
                                            0.9/ncols, 0.9/nrows,
                                            linewidth=0, edgecolor=cellcol, facecolor=cellcol)
                  ax.add_patch(rect)
          return fig

      def display(self):
          display(self.figure())

In [292]:
from copy import deepcopy
class SlidingBlocksPuzzle( SearchProblem ):

    def __init__( self, initial_state, goal, colors=COLORS ):
        """
        The __init__ method must set the initial state for the search.
        Arguments could be added to __init__ and used to configure the
        initial state and/or other aspects of a problem instance.
        """
        self.initial_state = BlockState(initial_state, colors=colors)
        self.colors = colors
        self.goal = BlockState(goal)

    def info(self):
        print("Solve the following sliding blocks problem.")
        print("Get from this initial state:")
        self.initial_state.display()
        print("To a state incorporating the following block position(s):")
        self.goal.display()
        print("You need to slide the red block to cover the bottom right square.")

    def possible_actions(self, state):
        return state.possible_moves()

    def successor(self, state, action):
        """
        This takes a state and an action and returns the new state resulting
        from doing that action in that state. You can assume that the given
        action is in the list of 'possible_actions' for that state.
        """
        return state.next_state(action)

    def goal_test(self, state):
        """
        For the sliding blocks puzzles, the goal condition is reached when
        all block possitions specified in the given goal state are satisfied by
        the current state. But empty positions (ie 0s) in the goal are ignored,
        so can be occupied by blocks in the current sate.
        """
        for row in range(state.nrows):
          for col in range(state.ncols):
            goalnum = self.goal.blockstate[row][col]
            if goalnum==0:
              continue
            if goalnum != state.blockstate[row][col]:
              return False
        return True


    def cost(self, path, state):
        """
        This is an optional method that you only need to define if you are using
        a cost based algorithm such as "uniform cost" or "A*". It should return
        the cost of reaching a given state via a given path.
        If this is not re-defined, it will is assumed that each action costs one unit
        of effort to perform, so it returns the length of the path.
        """
        return len(path)

    def display_action(self, action):
        """
        You can set the way an action will be displayed in outputs.
        """
        print((self.colors[action[0]], action[1]))

    def display_state(self, state):
        """
        You can set the way a state will be displayed in outputs.
        """
        state.display()

    def display_state_path( self, actions ):
        """
        This defines output of a solution path when a list of actions
        is applied to the initial state. It assumes it is a valid path
        with all actions being possible in the preceeding state.
        You probably don't need to override this.
        """
        s = self.initial_state
        self.display_state(s)
        for a in actions:
            self.display_action(a)
            s = self.successor(s,a)
            self.display_state(s)

## Part B

In [293]:
def parcel_heuristic(parcel):
    """Compute a score for each parcel to prioritize selection."""
    is_first_class = 1 if parcel.priority == 1 else 0
    is_stored_long = 1 if parcel.storage_days() >= 4 else 0
    return (10 * is_first_class) + (5 * is_stored_long)

def least_full_belt(conveyor_belts):
    return min(conveyor_belts, key=lambda cb: len(cb.items))

### Implementation

In [294]:
from copy import deepcopy
from datetime import datetime, timedelta
import random

# Generate random date within the last 7 days
def generate_random_date():
    days_ago = random.randint(1, 7)
    return datetime.now() - timedelta(days=days_ago)

class Parcel:
    def __init__(self, parcel_id, date, priority, size=None):
        self.parcel_id = parcel_id
        self.date = datetime.strptime(date, '%Y-%m-%d')  # Convert string to datetime object
        self.priority = priority  # 1 = First Class, 2 = Second Class
        self.size = size if size else random.randint(10, 20)  # Parcel size between 10% and 20%

    def __repr__(self):
        return f"({self.parcel_id}, {self.date.strftime('%Y-%m-%d')}, {self.priority}, Size: {self.size}%)"
    
    def storage_days(self):
        return (datetime.now() - self.date).days


class Robot:
    def __init__(self, location, carried_parcels=None):
        if carried_parcels is None:
            carried_parcels = []
        self.location = location
        self.carried_parcels = carried_parcels

    def __repr__(self):
        return f"Robot at {self.location}, Carrying: {self.carried_items}"

class ConveyorBelt:
    def __init__(self, name):
        self.name = name
        self.items = []
        self.initial_capacity = 100  # Start with 100% capacity
        self.capacity = 100  # Conveyor starts fully available

    def __repr__(self):
        return f"{self.name}: {self.items}, Capacity: {self.capacity}%"
    
    def add_parcel(self, parcel):
        """Add a parcel to the conveyor if there's enough space."""
        if self.capacity >= parcel.size:
            self.items.append(parcel)
            self.capacity -= parcel.size  # Reduce capacity based on parcel size
            return True  # Successfully added parcel
        else:
            print(f"{self.name} is full! Cannot add Parcel {parcel.parcel_id}.")
            return False  # Parcel could not be added
    

class State:
    def __init__(self, robot, conveyor_belts, storage_room):
        self.robot = robot
        self.conveyor_belts = conveyor_belts
        self.storage_room = storage_room

    def __repr__(self):
        return f"Robot: {self.robot}, Storage: {self.storage_room}, Belts: {self.conveyor_belts}"
    

class RobotWorker(SearchProblem):
    def __init__(self, state, goal_item_locations):
        self.initial_state = state
        self.goal_item_locations = goal_item_locations

    def possible_actions(self, state):
        next_state = deepcopy(state)
        actions = []

        # Define possible actions: picking up parcels, dropping off parcels, and moving between locations.
        if state.robot.location == 'storage room' and (state.robot.carried_parcels != None):
            for i in range(min(5, len(state.storage_room))):  # Limit to 5 parcels
                actions.append(("pick up", state.storage_room[i]))

        # Drop off parcels at conveyor belts
        for parcel in next_state.robot.carried_parcels:
            least_full = least_full_belt(state.conveyor_belts)
            if least_full.capacity > 0:
                actions.append(("drop off", parcel))  # Drop off parcel at the least full conveyor belt

        # Always move to the least full conveyor belt
        actions.append(("move to", least_full_belt(state.conveyor_belts).name))

        return actions

    def successor(self, state, action, actions):
        next_state = deepcopy(state)
        act, target = action
        actions = actions

        if act == "pick up":
            target_id = target.parcel_id
            for i in state.storage_room:
                if i.parcel_id == target_id:
                    state.storage_room.remove(i)
            state.robot.carried_parcels.append(target)
            actions.remove((act, target))
        
        elif act == "move to":
            if target == "storage room":
                next_state.robot.location = target
            else:
                for parcel in state.robot.carried_parcels:
                    state.robot.carried_parcels.remove(parcel)
                    for belt in state.conveyor_belts:
                        if target == belt.name:
                            belt.add_parcel(parcel)
                            print(f"Parcel {parcel.parcel_id} added to {belt.name}")
                next_state.robot.location = target
            actions.remove(action)
            actions.append(("move to", "storage room"))
        
        return next_state

    def goal_test(self, state):
        if len(state.storage_room) == 0 and all(len(belt.parcels) > 0 for belt in state.conveyor_belts):
            return True
        return False

    def display_state(self, state):
        print(f"Robot location: {state.robot.location}")
        print("Carried parcels:", [parcel.parcel_id for parcel in state.robot.carried_parcels])


In [295]:
STORAGE_ROOM = [
    Parcel(str(i), generate_random_date().strftime('%Y-%m-%d'), random.choice([1, 2]), random.randint(10, 20))
    for i in range(101, 119)
]

CONVEYOR_BELTS = [
    ConveyorBelt('conveyorA'),
    ConveyorBelt('conveyorB'),
    ConveyorBelt('conveyorC')
]

rob = Robot('storage room', [])
state = State(rob, CONVEYOR_BELTS, STORAGE_ROOM)
RW_PROBLEM = RobotWorker(state, {})

# sorted via heuristic
STORAGE_ROOM = sorted(STORAGE_ROOM, key=parcel_heuristic, reverse=True)  # Keep it as a list!

print("Initial Parcel Order:", [p.parcel_id for p in STORAGE_ROOM])

# poss_acts = RW_PROBLEM.possible_actions(RW_PROBLEM.initial_state)
# poss_acts

# print(poss_acts)
# for acts in poss_acts:
#     print(poss_acts)
#     next_state = RW_PROBLEM.successor(RW_PROBLEM.initial_state, acts,poss_acts)
#     RW_PROBLEM.display_state(next_state)
#     print()


for parcel in STORAGE_ROOM[:]:  # Iterate over a copy
    least_full = least_full_belt(CONVEYOR_BELTS)

    # Check if there is enough space on the belt
    if least_full.capacity >= parcel.size:
        print(f"\nMoving parcel {parcel.parcel_id} (Priority: {parcel.priority}, Size: {parcel.size}%, Date: {parcel.date}) to {least_full.name}")

        # Remove from storage
        STORAGE_ROOM.remove(parcel)

        # Add to belt
        least_full.add_parcel(parcel)
    else:
        print(f"!!! All conveyors are full or cannot fit Parcel {parcel.parcel_id} (Size: {parcel.size}%)!!! Keeping in storage.")

    # Print current conveyor status
    for belt in CONVEYOR_BELTS:
        print(f"{belt.name} contents: {[f'({p.parcel_id}, {p.date}, {p.priority}, {p.size}%)' for p in belt.items]} Capacity: {belt.capacity}%")

    # Print remaining parcels in storage
    print("Remaining in Storage:", [p.parcel_id for p in STORAGE_ROOM], "\n")

# output final state
print("\n\nFinal Parcel States:")
for belt in CONVEYOR_BELTS:
    print(f"{belt.name}: {[p.parcel_id for p in belt.items]}")
print("Remaining in Storage:", [p.parcel_id for p in STORAGE_ROOM],"\n")



Initial Parcel Order: ['101', '102', '106', '110', '117', '105', '108', '109', '115', '116', '103', '107', '113', '118', '104', '111', '112', '114']

Moving parcel 101 (Priority: 1, Size: 16%, Date: 2025-02-15 00:00:00) to conveyorA
conveyorA contents: ['(101, 2025-02-15 00:00:00, 1, 16%)'] Capacity: 84%
conveyorB contents: [] Capacity: 100%
conveyorC contents: [] Capacity: 100%
Remaining in Storage: ['102', '106', '110', '117', '105', '108', '109', '115', '116', '103', '107', '113', '118', '104', '111', '112', '114'] 


Moving parcel 102 (Priority: 1, Size: 15%, Date: 2025-02-17 00:00:00) to conveyorB
conveyorA contents: ['(101, 2025-02-15 00:00:00, 1, 16%)'] Capacity: 84%
conveyorB contents: ['(102, 2025-02-17 00:00:00, 1, 15%)'] Capacity: 85%
conveyorC contents: [] Capacity: 100%
Remaining in Storage: ['106', '110', '117', '105', '108', '109', '115', '116', '103', '107', '113', '118', '104', '111', '112', '114'] 


Moving parcel 106 (Priority: 1, Size: 11%, Date: 2025-02-17 00:00:00